<a href="https://colab.research.google.com/github/max-ostapenko/ga4_data_import/blob/main/GA4_Data_Import_from_Google_Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Some code here
import json
import os
import subprocess

DEBUG = False


def create_instance():
    """
    Returns:
        dict, https://cloud.google.com/compute/docs/reference/rest/v1/instances#resource:-instance
    """
    statis_ip_create_cmd = f"""
        gcloud compute addresses create {INSTANCE_NAME}-static \
            --project={GCP_PROJECT_ID} \
            --network-tier=STANDARD \
            --region={REGION} \
            --format='value(address)'
    """

    #static_ip = subprocess.check_output(statis_ip_create_cmd, shell=True).decode().strip()
    static_ip = "35.208.180.229"

    startup_script = f"""
#!/bin/bash

SFTP_USERNAME={SFTP_USERNAME}
BUCKET_NAME={BUCKET_NAME}

# Install SFTP server
apt-get update -y
apt-get install -y openssh-server

# Install gcloud
# https://cloud.google.com/sdk/docs/install#installation_instructions
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | tee /usr/share/keyrings/cloud.google.gpg
apt-get update -y
apt-get install google-cloud-sdk -y

# Install gcsfuse
# https://cloud.google.com/storage/docs/gcsfuse-quickstart-mount-bucket#install
export GCSFUSE_REPO=gcsfuse-$(lsb_release -c -s)
echo "deb https://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
sudo apt-get update -y
sudo apt-get install -y fuse gcsfuse
rm -rf /var/lib/apt/lists/*

# Create user
adduser $SFTP_USERNAME
mkdir /home/$SFTP_USERNAME
chown root:root /home/sftp_user
sed -i "s/^Subsystem\tsftp.*/Subsystem\tsftp internal-sftp/" /etc/ssh/sshd_config
tee -a /etc/ssh/sshd_config << EOM
Match User $SFTP_USERNAME
\tForceCommand internal-sftp -d /sftp
\tChrootDirectory /home/%u
\tAllowTcpForwarding no
\tX11Forwarding no
\tPasswordAuthentication no
\tAuthenticationMethods publickey
EOM
systemctl restart ssh

# Mount bucket
sudo -u $SFTP_USERNAME mkdir /home/$SFTP_USERNAME/sftp
sudo -u $SFTP_USERNAME gcsfuse $BUCKET_NAME /home/$SFTP_USERNAME/sftp"""
    with open('startup.sh', 'w') as file:
        file.write(startup_script)
    
    instance_create_cmd = f"""
    gcloud compute instances create {INSTANCE_NAME} \
        --project={GCP_PROJECT_ID} \
        --zone={ZONE} \
        --machine-type=f1-micro \
        --network-interface=address={static_ip},network-tier=STANDARD,subnet=default \
        --maintenance-policy=MIGRATE \
        --provisioning-model=STANDARD \
        --service-account={PROJECT_NUMBER}-compute@developer.gserviceaccount.com \
        --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append \
        --tags=default-allow-ssh \
        --create-disk=auto-delete=yes,boot=yes,image=projects/debian-cloud/global/images/debian-11-bullseye-v20230509,mode=rw,size=10,type=projects/max-ostapenko/zones/us-central1-a/diskTypes/pd-balanced \
        --no-shielded-secure-boot \
        --shielded-vtpm \
        --shielded-integrity-monitoring \
        --labels=ec-src=vm_add-gcloud \
        --reservation-affinity=any \
        --metadata-from-file=startup-script=startup.sh \
        --format=json
    """
    instance_info = json.loads(subprocess.check_output(instance_create_cmd, shell=True))[0]
    
    return instance_info


def add_shh_pub_key(key):
    """
    Args:
        key: SSH public key value

    """
    # Get the existing SSH keys from current instance metadata
    instance_describe_cmd = f"""
    gcloud compute instances describe {INSTANCE_NAME} \
        --project={GCP_PROJECT_ID} \
        --zone={ZONE} \
        --format='json(metadata)'
    """
    metadata = json.loads(subprocess.check_output(instance_describe_cmd, shell=True))

    existing_keys = ""
    for metadata_item in metadata.get("metadata",{}).get("items",{}):
        if metadata_item.get("key","") == "ssh-keys":
            existing_keys = metadata_item.get("value","")
            break
    
    new_key = f"{SFTP_USERNAME}:{key}".strip()
    need_append = True
    for key in existing_keys.splitlines():
        key = key.strip()
        if new_key == key:
            need_append = False
            break

    # Append the new public key to the VM SSH keys
    if DEBUG:
        print(f"""Existing keys:
{existing_keys}
New key:
{new_key}
    """)

    # Update the instance metadata with the new SSH keys
    if need_append:
        keys_file = "keys.txt"
        with open(keys_file, "w") as file:
            if existing_keys:
                file.write(f"{existing_keys}\n{new_key}")
            else:
                file.write(f"{new_key}")
        !gcloud compute instances add-metadata {INSTANCE_NAME} --project={GCP_PROJECT_ID} --zone={ZONE} --metadata-from-file=ssh-keys={keys_file}
        !rm keys.txt

## 1 Start with documentation
Read [the official documentation](https://support.google.com/analytics/answer/10071301) to have an idea about features and limitations of GA4 data import.


## 2 Create a Data Source in [GA4 Admin Panel](https://analytics.google.com/analytics/web/#/admin/data-import/)

0. Follow [the guide on configuring SFTP data source](https://support.google.com/analytics/answer/10071301#import)

In [ ]:
#@markdown 1. Get your SFTP server url by hitting RUN and deploying VM Instance
GCP_PROJECT_ID = 'max-ostapenko'  #@param {type: "string"}
ZONE = 'us-central1-a' #@param {type: "string"}
REGION = "-".join(ZONE.split("-")[:-1])
INSTANCE_NAME = 'sftp-server' #@param {type: "string"}
SFTP_USERNAME = 'sftp_user' #@param {type: "string"}

from google.colab import auth
auth.authenticate_user()
!gcloud config set project {GCP_PROJECT_ID}

[PROJECT_NUMBER] = !gcloud projects describe {GCP_PROJECT_ID} --format="value(projectNumber)" 

#@markdown 2. Enter the name of your Google Cloud Storage Bucket
BUCKET_NAME = 'sftp-server-ga4' #@param {type: "string"}

INSTANCE_IP = create_instance()["networkInterfaces"][0]["accessConfigs"][0]["natIP"]

# Give READ access to a bucket to default VM Service Account
!gcloud storage buckets add-iam-policy-binding gs://{BUCKET_NAME} \
    --member=serviceAccount:{PROJECT_NUMBER}-compute@developer.gserviceaccount.com \
    --role=roles/storage.objectViewer \
    --no-user-output-enabled

SFTP_URL = f"sftp://{INSTANCE_IP}/sftp/cost_data.csv"

print(f"""\nNow you can enter your SFTP server url:
    {SFTP_URL}
cost_data.csv file has be located in the root of the GCS Bucket.""")

In [ ]:
#@markdown 3. Paste and push Public Key from GA4 to SFTP server for authorisation
KEY_VALUE = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDeDrtHfPb8TM6/BqANwQJdSmAGjygLE+2jMdIxfJZFRdcEogMLMrEARhzj9fjRQKw7RhudOpxkyvMjVRXlv6zvcbvmJhQCxk6yp3Cgq3U7ux2PYppBWSxei1R4DH2TSjy/k/7nuKDvdDaBwUC/WLOgjvQ5nKLbYp53RbWoPrv63YEDDEyuLjMww9rlJ+J9iL/fd1TxlZxvwVvPCxweWCzfQfLP6MWGvDmUMfe7eTg11rt3p/wdNUa/RxKC1HOJRWloGCw7dX5LgZYoCq8tKOS8rpHt8BoTFhKjoPsIw8h4uNsjOb9y6YSOQrQFaXj3o2l4X0ssLxkKQcaG+EDZ8PtUr/WnuPpfHCuTNN1Q6TnaF2t2zm4J7C6LchPMgiP/vO+wwloQpgg9tUVGC4FZxQhHTLmCoW+19KIuBu4nm+zgaeIWXuHfnPLE4UdFsIB3o76tAfZprrykjaWVLPc6PFVlvRx1PIAPTL7dHaajnv1rxuQdQlNU2a2dOE4KNoxnffE= Google Analytics Data Import Key"  #@param {type: "string"}
add_shh_pub_key(KEY_VALUE)

LOCAL_KEY = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDIjG9uMbjXCMmJ6oLlI5rVlbm+eY2Ur42fCE542NAGwR64lEYmv2LkxPRzFnVU2mdF6+Ka4D8UyjM6D3fVkNfqQH8ckthcoMEgCNNK6tSU690I5YXAfGvCstGnljG4MwUWHkCFGXntqHPVnWjRfe0M88CusqazLy06VOr5hdfjTUXyxbyy4vx3kzBXE2oBXl0UZ33fI0/jHG3u6PULZ9kYu8jCXnNGp2b3ntgsMhitWTfj4GiSZF5wf6t4qDupgaW6OCrLJkgq9WS4OFdM4ZyHJadRff7KyHjsjtFoXzHOxhCrk4gj484HO86QyoA8jJE92o6k7OuVP9/Dl/w5n2jRtsvj0Fx2UJM3LH57XvflWWKOvfoYXl1v0Li78xnzLIgwDi282CHh31NNlbdm0Ux8AQXuQlLWd4R13sleLmdpqpcvrkCHktpvOIOvndar311CsPW3AvnHxNOKNAaJfE1UcP45AJ5TU6DrRJj9gAi+AJ/iET9W5XtKJOIk8JtHX6TwCXlSloZgPf+LOOscnYb5EQO93LCSyHQVwXuF35hV9DPFTXM+PUdhZCaPLwETjnoP8pQMYg1h8B6hCY9cXWlZQpxDyx4TUP/8vRuizX2oR5rLDG7p0P8x+IX6NtnuCQNockzl6DqXup/4o8SNVPHbC7Dv9uD9/zm9Pz7BK8TA+Q=="
add_shh_pub_key(LOCAL_KEY)

## 3 Test your SFTP server connention

1. Open [Google Cloud Shell terminal](https://shell.cloud.google.com/)

2. Authorise local SSH Key on the server
2. Connect to SFTP server
3. Show available files

In [ ]:
#@markdown Press RUN
if not os.path.isfile("/root/.ssh/id_rsa"):
    !ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N '' -q
[colab_sshpubkey] = !cat ~/.ssh/id_rsa.pub | awk '{print $1" "$2}'
add_shh_pub_key(colab_sshpubkey + " Google Colab")
!ssh-keygen -R {INSTANCE_IP}
!sftp -v {SFTP_USERNAME}@{INSTANCE_IP}